<a href="https://colab.research.google.com/github/heraldolimajr/Large-Language-Models/blob/main/Notebooks/BERT_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercício de Fine Tuning do BERT com PyTorch 🤖🐍🔥
Neste exercício, você irá implementar o fine tuning do BERT adicionando Task Head com finalidade de classificação. Sugestões utilizando a biblioteca PyTorch.

Conforme visto em sala, o Self-attention é um componente central dos Transformers, as redes neurais que impulsionam os modelos de linguagem modernos como o BERT. Após as camadas de atenção dos modelos, podemos adicionar uma ou mais camadas com a finalidade de executar tarefas específicas.

## Contexto
Um modelo BERT é um Transformer do tipo Encoder que processa um texto de entrada gerando representações semânticas desse texto. O Self-attention permite que o modelo determine a relação entre diferentes tokens em uma sequência.

Adicionaremos, conforme visto em sala de aula, Task Head ao modelo para fazer fine tuning de classificação.



---




### [<img src="https://colab.google/static/images/icons/colab.png" width=100> OPCIONAL ] Configuração do ambiente para melhor desempenho e instalação de dependências
💡 **Obs**: Selecione um ambiente com GPU para rodar esse notebook. No Google Colab:
**Runtime > Change runtime type > Hardware accelerator > GPU > GPU type > T4**.

---

In [1]:
# Caso esteja no Google Colab será necessário instalar apenas as dependências abaixo
!pip install seqeval>=1.2.2
!pip install evaluate>=0.4.0

# Importando dataset

Utilizaremos o dataset Rotten Tomatoes, que contém avaliações de filmes (https://huggingface.co/datasets/cornell-movie-review-data/rotten_tomatoes).

In [ ]:
from datasets import load_dataset

# Preparando dados
tomatoes = load_dataset("rotten_tomatoes")
# Separando
train_data, test_data = tomatoes["train"], tomatoes["test"]

In [3]:
# Vejamos o que tem aqui...
print(train_data)

print(train_data[:10])

print(train_data[-10:])

Dataset({
    features: ['text', 'label'],
    num_rows: 8530
})
{'text': ['the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .', 'the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson\'s expanded vision of j . r . r . tolkien\'s middle-earth .', 'effective but too-tepid biopic', 'if you sometimes like to go to the movies to have fun , wasabi is a good place to start .', "emerges as something rare , an issue movie that's so honest and keenly observed that it doesn't feel like one .", 'the film provides some great insight into the neurotic mindset of all comics -- even those who have reached the absolute top of the game .', 'offers that rare combination of entertainment and education .', 'perhaps no picture ever made has more literally showed that 

# Carregando o modelo BERT

Utilizaremos checkpoint "bert-base-uncased".

Carregaremos o tokenizer, conforme exercícios anteriores, mas também a classe `AutoModelForSequenceClassification`.

A classe `AutoModelForSequenceClassification` da biblioteca Hugging Face Transformers é uma classe automática projetada para simplificar o carregamento de modelos de Sequence Classification (Classificação de Sequência) já criando nossa Task Head.

Vejamos a diferença entre o carregamento do modelo original e o modelo com a Task Head Classification.

In [4]:
from transformers import AutoModel, AutoModelForSequenceClassification, AutoConfig
import torch.nn as nn

# Suprime warnings
import warnings
warnings.filterwarnings('ignore')

# O checkpoint base do BERT (sem ajuste fino para tarefa específica)
MODEL_CHECKPOINT = "bert-base-uncased"
QNT_CLASSES = 2  # Definimos 2 classes (ex: positivo, negativo)

print("--- 1. Carregando APENAS o Backbone BERT (BertModel) ---")
# AutoModel carrega o modelo base (o 'backbone' ou 'corpo' do transformer)
modelo_base = AutoModel.from_pretrained(MODEL_CHECKPOINT)

# Exibe a estrutura do modelo base
print(modelo_base)

print("\n" + "="*80 + "\n")

print(f"--- 2. Carregando BERT com a Cabeça de Classificação ({QNT_CLASSES} classes) ---")

# 2.1. Criar uma configuração para 2 classes
config = AutoConfig.from_pretrained(MODEL_CHECKPOINT, num_labels=QNT_CLASSES)

# 2.2. Carregar o modelo usando AutoModelForSequenceClassification
modelo_class = AutoModelForSequenceClassification.from_pretrained(
    MODEL_CHECKPOINT,
    config=config
)

# Exibe a estrutura do modelo de Classificação
print(modelo_class)

--- 1. Carregando APENAS o Backbone BERT (BertModel) ---


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

## Carregando novamente o modelo para utilização em nossa prática

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

modelo = AutoModelForSequenceClassification.from_pretrained(MODEL_CHECKPOINT, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

## 💭 Tokenizando a entrada

In [ ]:
from transformers import DataCollatorWithPadding

# Necessário para otimizar o padding no batch
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Definindo função de preprocessamento dos dados
def preprocessamento(itens):
   return tokenizer(itens["text"], truncation=True)

# Tokenizando dados train e test
tokenized_train = train_data.map(preprocessamento, batched=True)
tokenized_test = test_data.map(preprocessamento, batched=True)

## Definição de métricas

In [7]:
import numpy as np
import evaluate


def compute_metrics(eval_pred):
    """Calculate F1 score"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    load_f1 = evaluate.load("f1")
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"f1": f1}

# Treinamento

In [8]:
from transformers import TrainingArguments, Trainer

# Argumentos do treinamento
training_args = TrainingArguments(
   "model",
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=1,
   weight_decay=0.01,
   save_strategy="epoch",
   report_to="none"
)

# Instanciando o objeto "treinador"
treinador = Trainer(
   model=modelo,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)


In [ ]:
treinador.train()

## Avaliando os resultados obtidos

In [ ]:
treinador.evaluate()

## ...mas o que foi treinado afinal de contas?

Vamos exibir a configuração dos parâmetros para entender o que está acontecendo.

In [11]:
# Exibindo detalhes das camadas
for name, param in modelo.named_parameters():
    print(f"Parameter: {name} ----- {param.requires_grad}")

Parameter: bert.embeddings.word_embeddings.weight ----- True
Parameter: bert.embeddings.position_embeddings.weight ----- True
Parameter: bert.embeddings.token_type_embeddings.weight ----- True
Parameter: bert.embeddings.LayerNorm.weight ----- True
Parameter: bert.embeddings.LayerNorm.bias ----- True
Parameter: bert.encoder.layer.0.attention.self.query.weight ----- True
Parameter: bert.encoder.layer.0.attention.self.query.bias ----- True
Parameter: bert.encoder.layer.0.attention.self.key.weight ----- True
Parameter: bert.encoder.layer.0.attention.self.key.bias ----- True
Parameter: bert.encoder.layer.0.attention.self.value.weight ----- True
Parameter: bert.encoder.layer.0.attention.self.value.bias ----- True
Parameter: bert.encoder.layer.0.attention.output.dense.weight ----- True
Parameter: bert.encoder.layer.0.attention.output.dense.bias ----- True
Parameter: bert.encoder.layer.0.attention.output.LayerNorm.weight ----- True
Parameter: bert.encoder.layer.0.attention.output.LayerNorm.bia

# ⛄ 🥶 🧊❄ Congelamento de Camadas 🧊🧊🧊 ☃ ❄ 🥶

Uma tática muito comum para fine tuning é o congelamento de camadas, ou seja, bloquear a atualização dos pesos no treinamento da rede neural.

O 🐍PyTorch🔥 permite configurar explicitamente quais parâmetros podem ser atualizados. O atributo `requires_grad` é um boolean que controla se o parâmetro requer cálculo de gradiente. Ou seja, quando `requires_grad == False` o sistema de autograd do PyTorch não rastreia as operações que o envolvem, não calcula nem armazena seu gradiente durante o backpropagation, e, consequentemente, o otimizador não ajusta seu valor, mantendo-o constante ao longo de todo o processo de treinamento.

---

Vamos carregar novamente o modelo, mas agora vamos ajustar essa flag apenas nos parâmetros da camada de classificação.



In [13]:
# Load Model and Tokenizer
modelo_bert_congelado = AutoModelForSequenceClassification.from_pretrained(MODEL_CHECKPOINT, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

for name, param in modelo_bert_congelado.named_parameters():
    # Ajusta a camada "classifier"
    if name.startswith("classifier"):
      param.requires_grad = True
    else:
      param.requires_grad = False
    print(f"Parameter: {name} ----- {param.requires_grad}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Parameter: bert.embeddings.word_embeddings.weight ----- False
Parameter: bert.embeddings.position_embeddings.weight ----- False
Parameter: bert.embeddings.token_type_embeddings.weight ----- False
Parameter: bert.embeddings.LayerNorm.weight ----- False
Parameter: bert.embeddings.LayerNorm.bias ----- False
Parameter: bert.encoder.layer.0.attention.self.query.weight ----- False
Parameter: bert.encoder.layer.0.attention.self.query.bias ----- False
Parameter: bert.encoder.layer.0.attention.self.key.weight ----- False
Parameter: bert.encoder.layer.0.attention.self.key.bias ----- False
Parameter: bert.encoder.layer.0.attention.self.value.weight ----- False
Parameter: bert.encoder.layer.0.attention.self.value.bias ----- False
Parameter: bert.encoder.layer.0.attention.output.dense.weight ----- False
Parameter: bert.encoder.layer.0.attention.output.dense.bias ----- False
Parameter: bert.encoder.layer.0.attention.output.LayerNorm.weight ----- False
Parameter: bert.encoder.layer.0.attention.output

## Treinando o modelo carregado novamente

Agora será executado o processo de treinamento e vejamos a diferença.

In [ ]:
from transformers import TrainingArguments, Trainer

treinador = Trainer(
   model=modelo_bert_congelado,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

treinador.train()

## Nova Avaliação do treinamento 🏋

In [ ]:
treinador.evaluate()

In [ ]:
from datasets import concatenate_datasets

# Pegar exemplos do dataset de teste (mistura de positivos e negativos)
positive_reviews = test_data.filter(lambda example: example['label'] == 1).select(range(5))
negative_reviews = test_data.filter(lambda example: example['label'] == 0).select(range(5))

# Combinar os exemplos
sample_reviews = concatenate_datasets([positive_reviews, negative_reviews])


# Preprocessar as avaliações individualmente
tokenized_sample_list = [tokenizer(review, truncation=True) for review in sample_reviews['text']]

# Usar o data collator para preparar a entrada para o modelo
import torch
batch = data_collator(tokenized_sample_list)

# Mover o batch para o mesmo dispositivo do modelo
batch = {k: v.to(modelo_bert_congelado.device) for k, v in batch.items()}


# Fazer previsões
with torch.no_grad():
    outputs = modelo_bert_congelado(**batch)
    predictions = torch.argmax(outputs.logits, dim=-1)

# Exibir os resultados
for i, review in enumerate(sample_reviews['text']):
    predicted_label = "Positive" if predictions[i].item() == 1 else "Negative"
    # label real para fins de comparação
    actual_label = "Positive" if sample_reviews['label'][i] == 1 else "Negative"
    print(f"Review: {review}\nPredicted Label: {predicted_label}\nActual Label: {actual_label}\n")

## Para testar o classificador com uma string sua, utilize o código abaixo:

Lembre-se de enviar a variável para a GPU, para não ver nenhum erro bizarro 😅.

In [17]:
texto_teste = "This is a horrible movie"

# Executa inferência no modelo treinado
# Mudando para o modo eval
modelo.eval()

entrada_tokenizada = tokenizer(texto_teste, return_tensors="pt")

# Move os tensores para o dispositivo em que o modelo se encontra
device = modelo.device
entrada_tokenizada = {k: v.to(device) for k, v in entrada_tokenizada.items()}

with torch.no_grad():
  saida = modelo_bert_congelado(**entrada_tokenizada)

print(f"Saída bruta: {saida}")

# Trabalhando a saída bruta
logits = saida.logits

prob = torch.softmax(logits, dim=1)
print(f"Probabilidades: {prob}")

Saída bruta: SequenceClassifierOutput(loss=None, logits=tensor([[ 0.2159, -0.0121]], device='cuda:0'), hidden_states=None, attentions=None)
Probabilidades: tensor([[0.5568, 0.4432]], device='cuda:0')



---

# Tarefas do Exercício
## 1. Agora responda com suas palavras o que aconteceu em ambos os casos durante o treinamento, evidenciando se você percebeu alguma diferença durante o passo de treinamento. Discuta brevemente os resultados obtidos.

In [ ]:
# No segundo caso (modelo congelado), o treinamento é mais leve e estável, mas limitado, sendo eficiente quando o conjunto de dados é pequeno
# ou quando a tarefa é semelhante ao que o BERT já sabe. No primeiro (modelo descongelado), o modelo aprende mais profundamente, o que normalmente
# leva a desempenho superior, porém com maior custo de tempo e complexidade de treinamento.

## 2. Conforme instruções anteriores, carregue mais uma vez o modelo BERT, mas agora mantenha apenas a partir da camada encoder 10 (`bert.encoder.layer.10`) do modelo BERT como treinável (`require_grad == True`). Ou seja, congele (`requires_grad == False`) até a camada encoder 9 (`bert.encoder.layer.9`).

In [26]:
# Load Model and Tokenizer
modelo_bert_congelado2 = AutoModelForSequenceClassification.from_pretrained(MODEL_CHECKPOINT, num_labels=2)
tokenizer2 = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

# 1) Congelar TODAS as camadas inicialmente
for param in modelo_bert_congelado2.bert.parameters():
    param.requires_grad = False

# 2) Descongelar a partir da camada 10 (10 e 11)
for param in modelo_bert_congelado2.bert.encoder.layer[10:].parameters():
    param.requires_grad = True

# (Opcional) também descongelar o pooler
for param in modelo_bert_congelado2.bert.pooler.parameters():
    param.requires_grad = True

# O classificador final já vem descongelado por padrão
for param in modelo_bert_congelado2.classifier.parameters():
    param.requires_grad = True

#testa se deu certo
for name, param in modelo_bert_congelado2.named_parameters():
  if "encoder.layer.9" in name:
    print(name, "->", param.requires_grad)
  if "encoder.layer.10" in name:
    print(name, "->", param.requires_grad)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


bert.encoder.layer.9.attention.self.query.weight -> False
bert.encoder.layer.9.attention.self.query.bias -> False
bert.encoder.layer.9.attention.self.key.weight -> False
bert.encoder.layer.9.attention.self.key.bias -> False
bert.encoder.layer.9.attention.self.value.weight -> False
bert.encoder.layer.9.attention.self.value.bias -> False
bert.encoder.layer.9.attention.output.dense.weight -> False
bert.encoder.layer.9.attention.output.dense.bias -> False
bert.encoder.layer.9.attention.output.LayerNorm.weight -> False
bert.encoder.layer.9.attention.output.LayerNorm.bias -> False
bert.encoder.layer.9.intermediate.dense.weight -> False
bert.encoder.layer.9.intermediate.dense.bias -> False
bert.encoder.layer.9.output.dense.weight -> False
bert.encoder.layer.9.output.dense.bias -> False
bert.encoder.layer.9.output.LayerNorm.weight -> False
bert.encoder.layer.9.output.LayerNorm.bias -> False
bert.encoder.layer.10.attention.self.query.weight -> True
bert.encoder.layer.10.attention.self.query.bia

## 3. Execute o treinamento com os mesmos parâmetros utilizados anteriormente e execute o método `evaluate()` para calcular as métricas e discuta os resultados obtidos.
Caso julgar necessário, execute outros treinamentos ajustando quantidades distintas de parâmetros treináveis para tirar conclusões adicionais.

In [ ]:
from transformers import TrainingArguments, Trainer

treinador = Trainer(
   model=modelo_bert_congelado2,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

treinador.train()

In [ ]:
treinador.evaluate()

## 4. (BÔNUS) Pesquise e escolha outro dataset para fazer fine tuning do modelo BERT.

In [ ]:
from datasets import load_dataset

#Dataset SST-2 (Stanford Sentiment Treebank 2)
#Tarefa: Sentiment Analysis (positivo/negativo)
#Tamanho: ~70k exemplos
#Fonte: GLUE Benchmark
#Domínio: Frases de reviews de filmes

sst2 = load_dataset("glue", "sst2")
ds_train = sst2["train"]        # ~67k
ds_valid = sst2["validation"]   # ~872
ds_test  = sst2["test"]         # sem labels

len(ds_train), len(ds_valid), len(ds_test)


In [ ]:
from transformers import BertTokenizerFast

pretrained = "bert-base-uncased"
tokenizer = BertTokenizerFast.from_pretrained(pretrained)

MAX_LEN = 128

def tokenize(batch):
    return tokenizer(
        batch["sentence"],
        truncation=True,
        padding=False,   # padding dinâmico via DataCollator
        max_length=MAX_LEN
    )

ds_train_tok = ds_train.map(tokenize, batched=True, remove_columns=["sentence", "idx"])
ds_valid_tok = ds_valid.map(tokenize, batched=True, remove_columns=["sentence", "idx"])
ds_test_tok  = ds_test.map(tokenize,  batched=True, remove_columns=["sentence", "idx"])

ds_train_tok = ds_train_tok.rename_column("label", "labels")
ds_valid_tok = ds_valid_tok.rename_column("label", "labels")
# ds_test não tem "label"

ds_train_tok.set_format("torch")
ds_valid_tok.set_format("torch")
ds_test_tok.set_format("torch")


In [35]:
# Load Model and Tokenizer
modelo_bert_congelado3 = AutoModelForSequenceClassification.from_pretrained(MODEL_CHECKPOINT, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

# 1) Congelar TODAS as camadas inicialmente
for param in modelo_bert_congelado3.bert.parameters():
    param.requires_grad = False

# 2) Descongelar a partir da camada 10 (10 e 11)
for param in modelo_bert_congelado3.bert.encoder.layer[10:].parameters():
    param.requires_grad = True

# (Opcional) também descongelar o pooler
for param in modelo_bert_congelado3.bert.pooler.parameters():
    param.requires_grad = True

# O classificador final já vem descongelado por padrão
for param in modelo_bert_congelado3.classifier.parameters():
    param.requires_grad = True

#testa se deu certo
for name, param in modelo_bert_congelado3.named_parameters():
  if "encoder.layer.9" in name:
    print(name, "->", param.requires_grad)
  if "encoder.layer.10" in name:
    print(name, "->", param.requires_grad)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


bert.encoder.layer.9.attention.self.query.weight -> False
bert.encoder.layer.9.attention.self.query.bias -> False
bert.encoder.layer.9.attention.self.key.weight -> False
bert.encoder.layer.9.attention.self.key.bias -> False
bert.encoder.layer.9.attention.self.value.weight -> False
bert.encoder.layer.9.attention.self.value.bias -> False
bert.encoder.layer.9.attention.output.dense.weight -> False
bert.encoder.layer.9.attention.output.dense.bias -> False
bert.encoder.layer.9.attention.output.LayerNorm.weight -> False
bert.encoder.layer.9.attention.output.LayerNorm.bias -> False
bert.encoder.layer.9.intermediate.dense.weight -> False
bert.encoder.layer.9.intermediate.dense.bias -> False
bert.encoder.layer.9.output.dense.weight -> False
bert.encoder.layer.9.output.dense.bias -> False
bert.encoder.layer.9.output.LayerNorm.weight -> False
bert.encoder.layer.9.output.LayerNorm.bias -> False
bert.encoder.layer.10.attention.self.query.weight -> True
bert.encoder.layer.10.attention.self.query.bia

In [ ]:
from transformers import TrainingArguments, Trainer

treinador3 = Trainer(
   model=modelo_bert_congelado3,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

treinador.train()

In [ ]:
treinador3.evaluate()